<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#导入分词词典、停用词词典" data-toc-modified-id="导入分词词典、停用词词典-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入分词词典、停用词词典</a></span></li><li><span><a href="#导入文本数据并分词" data-toc-modified-id="导入文本数据并分词-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入文本数据并分词</a></span></li><li><span><a href="#主题个数选择：基于困惑度和一致性" data-toc-modified-id="主题个数选择：基于困惑度和一致性-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>主题个数选择：基于困惑度和一致性</a></span></li><li><span><a href="#lda结果可视化" data-toc-modified-id="lda结果可视化-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>lda结果可视化</a></span></li></ul></div>

In [1]:
# coding:utf-8
import os
import csv
import jieba
import jieba.analyse
import pandas as pd
import warnings
warnings.filterwarnings('ignore') 

# 在gensim包中使用LDA
from gensim import corpora, models 
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

## 导入分词词典、停用词词典

In [2]:
# 载入自定义的分词词典
jieba.load_userdict(os.path.join('./model_data', "jiebaDict.txt"))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\15957\AppData\Local\Temp\jieba.cache
Loading model cost 0.802 seconds.
Prefix dict has been built successfully.


In [3]:
# 读取停用词表并去重
def get_stop_words_set(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        return set([line.strip() for line in file])

## 导入文本数据并分词

In [4]:
# 待分析数据去除停用词
def get_words_list(file_name, index, stop_word_file):
    '''
    filename:待分析数据文件名
    index:待分析数据所在列
    stop_word_file:停用词文件名
    '''
    stop_words_set = get_stop_words_set(stop_word_file)
    print("共导入 %d 个停用词" % len(stop_words_set))
    
    word_list = []
    with open(file_name, 'r', encoding = 'utf-8', errors = 'ignore') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            tmp_list = list(jieba.cut(line[index - 1], cut_all = False))
            tmplist = []
            for tmp in tmp_list:
                tmplist.append(tmp.upper())
            word_list.append([term for term in tmplist if str(term) not in stop_words_set]) 
    return word_list

In [5]:
# 文本列表，其中每个元素也是一个列表，即每行文字分词后形成的词语列表
word_list = get_words_list('./model_data/51job_describ.csv', 9,  './model_data/stop_words.txt') # 第9列:岗位要求
print(word_list[3])

# # 保存分词结果
# test = pd.DataFrame(data = [word_list])
# test.to_csv('./testcsv.csv', encoding = 'utf_8_sig')

共导入 2786 个停用词
['二十六岁', '大专', '学历', '驾驶证者', '优先考虑', '统计学', '项目管理', '计算机', '相关专业', '优先', '对系统', '搭建', '数据管理', '兴趣', 'CRM', '系统', '运维', '经验', '优先', '具备', '应变能力', '承压', '能力', '思维', '敏捷']


## 主题个数选择：基于困惑度和一致性

In [6]:
# 根据文本列表创建一个语料库，每个词与一个整型索引值对应
word_dict = corpora.Dictionary(word_list) 
# 词频统计，转化成空间向量格式
corpus_list = [word_dict.doc2bow(text) for text in word_list] 

In [7]:
import gensim.corpora 
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel

model_list = []
perplexity = [] # 困惑度 
coherence_values = [] # 一致性

for num_topics in range(2, 8, 1):    
    lda_model = models.LdaModel(corpus = corpus_list, id2word = word_dict, random_state = 1, num_topics = num_topics, passes = 20, alpha='auto')    
    model_list.append(lda_model) # 不同主题个数下的lda模型

    # 模型对应的困惑度（越低越好）
    perplexity_values = lda_model.log_perplexity(corpus_list)  
    print('第 %d 个主题的Perplexity为: ' % (num_topics - 1), round(perplexity_values, 3))   
    perplexity.append(round(perplexity_values, 3))

    # 模型对应的一致性（越高越好）    
    coherencemodel = CoherenceModel(model = lda_model, corpus = common_corpus, coherence = 'u_mass')    
    coherence_values.append(round(coherencemodel.get_coherence(),3))    
    print('第 %d 个主题的Coherence为: ' % (num_topics - 1), round(coherencemodel.get_coherence(),3))

第 1 个主题的Perplexity为:  -6.257
第 1 个主题的Coherence为:  -9.157
第 2 个主题的Perplexity为:  -6.239
第 2 个主题的Coherence为:  -9.49
第 3 个主题的Perplexity为:  -6.228
第 3 个主题的Coherence为:  -7.588
第 4 个主题的Perplexity为:  -6.239
第 4 个主题的Coherence为:  -6.843
第 5 个主题的Perplexity为:  -6.247
第 5 个主题的Coherence为:  -6.582
第 6 个主题的Perplexity为:  -6.247
第 6 个主题的Coherence为:  -6.255


In [10]:
# 最大的一致性值以及对应的主题个数
print('最大的Coherence为：' + str(max(coherence_values))) 
for i in range(len(coherence_values)):
    if coherence_values[i] == max(coherence_values):
        print('对应的主题个数为：' + str(i + 2))

-6.255
7


## lda结果可视化

In [11]:
import pyLDAvis.gensim_models

In [12]:
# 最终模型
## corpus: 文档词频矩阵
## num_topics：主题数目
## passes：训练伦次
lda = models.LdaModel(corpus = corpus_list, id2word = word_dict, random_state = 1, num_topics = 7, passes = 20, alpha='auto')

In [13]:
# 结果展示
## lda: 训练好的模型
d = pyLDAvis.gensim_models.prepare(lda, corpus_list, word_dict, mds = 'pcoa', sort_topics = True)

pyLDAvis.save_html(d, 'lda_show.html') # 将结果保存为html文件

# 展示在notebook的output cell中 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus_list, word_dict)
vis

C:\Users\15957\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.143471  0.063287       1        1  32.971553
3      0.115050  0.177284       2        1  29.576777
4      0.124292 -0.174216       3        1  12.643555
0      0.094387 -0.157041       4        1  12.287698
2     -0.015939  0.095631       5        1   8.557362
5     -0.160848  0.050434       6        1   3.238854
1     -0.300412 -0.055379       7        1   0.724201, topic_info=     Term         Freq        Total Category  logprob  loglift
24     数据  3017.000000  3017.000000  Default  30.0000  30.0000
8      熟悉  2325.000000  2325.000000  Default  29.0000  29.0000
110    开发   931.000000   931.000000  Default  28.0000  28.0000
11     经验  4082.000000  4082.000000  Default  27.0000  27.0000
178    分析  1713.000000  1713.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
739    有过     2.847180    45.876906   Topic7  -5.7592   2.1482
902  组织协调     2.545436    45.195819   Topic7  -5.8712   2.0512
241    心态     2.431753    24.433262   Topic7  -5.9169   2.6205
273    计算     2.587358    96.753971   Topic7  -5.8549   1.3063
521    电商     2.636650   236.404643   Topic7  -5.8360   0.4318

[533 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3603      6  0.911395       AD
5598      6  0.926620     ADAS
172       4  0.982328       AI
726       4  0.887941  ANDROID
1919      1  0.921093    AXURE
...     ...       ...      ...
926       6  0.041713       驱动
1164      4  0.958125       验证
2918      2  0.924298      高强度
1388      5  0.329028      高质量
1388      6  0.592251      高质量

[1062 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 5, 1, 3, 6, 2])